# ESnet SNMP REST API Connector

This notebook demonstrates how to use the `esnet_snmp` connector to retrieve ESnet traffic data from the ESnet SNMP REST API.

In [ ]:
import datetime

In [ ]:
import tokio.connectors.esnet_snmp

In [ ]:
ROUTER = 'sunn-cr5'
INTERFACE = 'to_nersc_ip-d_v4'
TARGET_DATE = datetime.datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=1)

In [ ]:
# Because the ESnet API treats the end date as inclusive, we subtract
# one second to avoid counting the first measurement of the following day.
esnetsnmp = tokio.connectors.esnet_snmp.EsnetSnmp(
    start=TARGET_DATE,
    end=TARGET_DATE + datetime.timedelta(days=1, seconds=-1)
)

In [ ]:
# Retrieve data in and out of the network interface and sanity check the timestamps
for direction in 'in', 'out':
    raw_return = esnetsnmp.get_interface_counters(endpoint=ROUTER,
                                                  interface=INTERFACE,
                                                  direction=direction,
                                                  agg_func='average')
    if not raw_return.get('data'):
        print("No data returned.  Issuing too many queries?  Or is date too far in the past?")
    else:
        print("Earliest data point for %3s: %s" % (direction, str(datetime.datetime.fromtimestamp(min(esnetsnmp[ROUTER][INTERFACE][direction].keys())))))
        print("Latest data point for %3s:   %s" % (direction, str(datetime.datetime.fromtimestamp(max(esnetsnmp[ROUTER][INTERFACE][direction].keys())))))

In [ ]:
bidirectional_traffic = 0
for direction in 'in', 'out':
    bytes_per_sec = list(esnetsnmp[ROUTER][INTERFACE][direction].values())
    total_bytes = sum(bytes_per_sec) * esnetsnmp.timestep
    bidirectional_traffic += total_bytes
    print("%s:%s saw %.2f TiB %s" % (
        ROUTER,
        INTERFACE,
        total_bytes / 2**40,
        direction))
print("Total bidirectional traffic: %.2f" % (bidirectional_traffic / 2**40))